In [1]:
#| default_exp music

In [2]:
#| export
from fasthtml.common import *
from fasthtml.components import Uk
from fh_frankenui.components import *
from fasthtml.components import Uk_icon
from fasthtml.svg import *
from fh_matplotlib import matplotlib2fasthtml
import numpy as np
from pathlib import Path
import matplotlib.pylab as plt

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">
<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
.disabled {opacity: 0.5; pointer-events: none; cursor: default; text-decoration: none;}
</style>

In [5]:
from httpx import get
fh_fui = Path('../fh_frankenui')
fh_fui_ex = Path('../ex_app')

fh_frankenui = (fh_fui / "components.py").read_text()

url = 'https://raw.githubusercontent.com/franken-ui/examples/master/src/pages/examples/music.astro'
music = get(url).text

url = 'https://raw.githubusercontent.com/franken-ui/examples/master/src/pages/examples/cards.astro'
cards = get(url).text

cards_in_python = (fh_fui_ex / 'cards.py').read_text()

url = 'https://raw.githubusercontent.com/AnswerDotAI/fasthtml/main/examples/adv_app.py'
idiomatic_fasthtml_app = get(url).text

In [6]:
%%ai 0 -c

I am implementing FrankenUI for FastHTML by reimplementing the FrankenUI examples from their docs in a python library.

I am doing this by building a library as I go, $`fh_frankenui`.  You should use this library as much as possible

Reference the $`idiomatic_fasthtml_app` to get a feel for fasthtml.

I previously implemented the $`cards` example, and the python implementation looked like $`cards_in_python`.

Now, I am looking to do the same with the $`music` example.

The goal is to develop the work-in-progress `fh_frankenui` library, so please use `fh_frankenui` as much as makes sense and recommend functions or improvements to `fh_frankenui` when it fits.

Please read all this information and confirm that you understand what I want to do by giving me a brief overview in your own words.

Before answering, let me know if any of the variables I supplied are invalid or empty.

I've reviewed the provided information and variables. All the variables appear to be valid and contain the expected content. Now, I'll provide a brief overview of your task as I understand it:

You're developing a Python library called `fh_frankenui` to implement FrankenUI components for FastHTML. Your approach is to recreate the FrankenUI examples from their documentation in Python, building up the library as you go. You've already implemented the 'cards' example, and now you're moving on to the 'music' example.

Your goals are:

1. Implement the 'music' example from FrankenUI in Python using FastHTML.
2. Utilize the existing `fh_frankenui` library as much as possible.
3. Suggest improvements or additional functions for `fh_frankenui` where appropriate.
4. Follow the idiomatic FastHTML style, referencing the `idiomatic_fasthtml_app` as a guide.

The process involves translating the HTML and JavaScript components from the FrankenUI examples into equivalent Python code using FastHTML's syntax and components. As you work through this, you're aiming to create reusable components and functions that can be added to the `fh_frankenui` library for future use.

Is this understanding correct? Would you like me to proceed with implementing the 'music' example or do you need any clarification?

In [7]:
#| export
def NavP(*c, cls=TextT.muted_sm): return P(cls=cls)(*c)

def SpacedPP(left, right=None):
    return FullySpacedContainer(NavP(left),NavP(right) if right else '')

def SpacedPPs(*c):
    return [SpacedPP(*tuplify(o)) for o in c]

def SpacedTxtIcon(txt, icon, ratio, icon_right=True):
    c = (NavP(txt),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return FullySpacedContainer(*c)  

def LAlignedTxtIcon(txt, icon='play-circle', ratio=1, icon_right=True, txt_cls=None):
    # TODO: Test and make sure this works
    # TODO: How do you pass text with a color? Should txt be a string?  Or should you pass a P?  
    c = (txt if isinstance(txt, FT) else NavP(txt,cls=ifnone(txt_cls,TextT.muted_sm)),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return Div(cls='flex items-center space-x-6')(*c)

In [56]:
#| export

# .disabled {opacity: 0.5; pointer-events: none; cursor: default; text-decoration: none;}
music_dd_data = [SpacedPPs("About Music",("Preferences", "⌘")),
                 SpacedPPs(("Hide Music", "⌘H"),("Hide Others", "⇧⌘H"),("Quit Music", "⌘Q"))]

# disabled = Burn Playlist to Disc | Print
file_dd_data = [SpacedPPs("New" ,("Open Stream URL", "⌘U"),("Close Window", "⌘W")),
                SpacedPPs("Library",("Import", "⌘O"),("Burn Playlist to Disc", )),
                SpacedPPs(("Show in Finder", "⇧⌘R"),"Convert"),
                SpacedPPs("Page Setup","Print")]

# disabled = Undo, Redo, Cut, Copy, Paste, Deselect All
edit_dd_data = [SpacedPPs(("Undo", "⌘Z"),("Redo", "⇧⌘Z")), SpacedPPs(("Cut", "⌘X"),("Copy", "⌘C"),("Paste", "⌘V")),
                SpacedPPs(("Select All", "⌘A"),("Deselect All", "⇧⌘A")),
                SpacedTxtIcon('Smart Dictation','microphone',0.5), SpacedTxtIcon("Emojis & Symbols",'world',0.5)]

#disabled = Show Status Bar | Enter Full Screen
view_dd_data = [SpacedPPs("Show Playing Next", "Show Lyrics", "Show Status Bar", "Hide Sidebar", "Enter Full Screen")]

# header: Switch account
account_dd_data = [Span("Switch Account", cls="ml-6"), [SpacedPP("Andy"), LAlignedTxtIcon("Benoit", 'plus-circle', 0.5, icon_right=False), SpacedPP("Luis")],
                   SpacedPPs("Manage Family"), SpacedPPs("Add Account")]

In [57]:
#| export

def wrap_in_a(item, has_header=False):
    if isinstance(item, (list, tuple)): return [s if i == 0 and has_header else wrap_in_a(s) for i, s in enumerate(item)]
    else: return item if has_header else A(item, href='#')

music_headers = UkNavbar(
    lnav=(
        UkNavbarDropdown(*wrap_in_a(music_dd_data), label='Music'),
        UkNavbarDropdown(*wrap_in_a(file_dd_data), label='File'),
        UkNavbarDropdown(*wrap_in_a(edit_dd_data), label='Edit'),
        UkNavbarDropdown(*wrap_in_a(view_dd_data), label='View'),
        UkNavbarDropdown(*wrap_in_a(account_dd_data, has_header=True), label='Account', has_header=True)
    ),
    cls='p-2'
)

In [58]:
#| export
_fn = lambda x: A(role='button', uk_toggle=True)(LAlignedTxtIcon(*x,icon_right=False,txt_cls=''))
discover = (UkH4("Discover"), *map(_fn, (("Listen Now", "play-circle"), ("Browse", 'thumbnails'), ("Radio", "rss"))))
library = (UkH4("Library"), *map(_fn, (("Playlists", "play-circle"), ("Songs", "bell"), ("Made for You", "user"),("Artists", "users"),("Albums", "bookmark"))))
_fn = lambda x: A(role='button', uk_toggle=True)(LAlignedTxtIcon(x,icon_right=False,txt_cls=''))
playlist = (UkH4("Playlist"), *map(_fn, ("Recently Added", "Recently Played", "Top Songs", "Top Albums", "Top Artists", 
                                         "Logic Discography","Bedtime Beats", "I miss Y2K Pop")))

In [59]:
#| export
def UkNav(*lis,cls='space-y-2'): return Ul(cls="uk-nav uk-nav-secondary "+cls)(*map(Li,lis))

In [60]:
#| export
def UkGrid(*c, cols, cls=''):
    gap_class = 'gap-x-4'  # This matches the gap in correct_grid
    grid_class = f"grid grid-cols-{cols}"
    full_class = ' '.join(filter(None, [grid_class, gap_class, cls]))
    return Div(cls=full_class)(*c)

In [61]:
#| export
def AlbumImg(url):
    return Div(cls="overflow-hidden rounded-md")(Img(cls="transition-transform duration-200 hover:scale-105", src=url))

def AlbumFooter(title, artist):
    return Div(cls='space-y-1')(P(title,cls=TextB.wt_bold),P(artist,cls=TextB.cl_muted))

def Album(url,title,artist):
    return Div(AlbumImg(url),AlbumFooter(title,artist))

In [62]:
#| export
listen_now_albums = (("Roar", "Catty Perry"), ("Feline on a Prayer", "Cat Jovi"),("Fur Elise", "Ludwig van Beethovpurr"),("Purrple Rain", "Prince's Cat"))

made_for_you_albums = [("Like a Feline", "Catdonna"),("Livin' La Vida Purrda", "Ricky Catin"),("Meow Meow Rocket", "Elton Cat"),
        ("Rolling in the Purr", "Catdelle",),("Purrs of Silence", "Cat Garfunkel"),("Meow Me Maybe", "Carly Rae Purrsen"),]
    

In [63]:
#| export
def create_album_grid(albums, cols=4):    
    return Div(cls=f"grid grid-cols-{cols} gap-x-4")(
        *[Div(cls="space-y-3")(
            Div(cls="overflow-hidden rounded-md")(
                Img(cls="transition-transform duration-200 hover:scale-105", src=img_url, alt="")
            ),
            Div(cls="space-y-1 text-sm")(
                H3(album['title'], cls="font-medium leading-none"),
                P(album['artist'], cls="text-xs text-muted-foreground")
            )
        ) for album in albums]
    )


In [64]:
#| export
_album = lambda t,a: Album('https://ucarecdn.com/e5607eaf-2b2a-43b9-ada9-330824b6afd7/music1.webp',t,a)

music_content = (Div(UkH3("Listen Now"), cls="mt-6 space-y-1"),
                    P("Top picks for you. Updated daily.",cls=TextT.muted_sm),
                    UkHLine(),
                    UkGrid(*[_album(t,a) for t,a in listen_now_albums], cols=4),
                    Div(UkH3("Made for You"), cls="mt-6 space-y-1"),
                    P("Your personal playlists. Updated daily.", cls=TextT.muted_sm),
                    UkHLine(),
                    UkGrid(*[_album(t,a) for t,a in made_for_you_albums], cols=6))

In [65]:
#| export
tabs = Ul(Li(A('Music', href='#'),cls='uk-active'),
    Li(A('Podcasts', href='#')),
    Li(A('Live', cls='opacity-50'), cls='uk-disabled'),
    uk_switcher='connect: #component-nav; animation: uk-animation-fade',
    cls='uk-tab-alt')

In [66]:
#| export
def podcast_tab():
    return Div(
        Div(cls="space-y-3")(
            UkH3("New Episodes"),
            P("Your favorite podcasts. Updated daily.", cls=TextT.muted_sm)),
        Div(cls="my-4 h-[1px] w-full bg-border"),
        Div(cls="uk-placeholder flex h-[450px] items-center justify-center rounded-md",uk_placeholder=True)(
            Div(cls="text-center space-y-6")(
                UkIcon(" microphone", 3),
                UkH4("No episodes added"),
                P("You have not added any podcasts. Add one below.", cls=TextT.muted_sm),
                UkButton("Add Podcast", cls=UkButtonT.primary))))

In [67]:
#| export
def page():
    return Div(music_headers,UkHSplit(),
        Div(cls="grid grid-cols-5")(
            Div(cls="hidden space-y-10 py-4 lg:block p-2")(
                UkNav(*discover),
                UkNav(*library),
                UkNav(*playlist)),
            Div(cls="col-span-5 border-l border-border lg:col-span-4")(
                Div(cls="px-8 py-6")(
                    Div(cls="flex items-center justify-between")(
                        Div(cls="max-w-80")(tabs),
                        UkButton(cls=UkButtonT.primary)(Span(cls="mr-2 size-4")(UkIcon('plus-circle', 0.8)),"Add music")),
                    Ul(id="component-nav", cls="uk-switcher")(
                        Li(*music_content),
                        Li(podcast_tab()))))))

In [68]:
#| export
music_homepage = page()

In [69]:
#| hide
import nbdev; nbdev.nbdev_export()